In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

import DistanceLib
import TreeLib
import ClusterLib

In [3]:
def calculate_wss(n, chosen_trees, centers):
    wss = 0
    for k in range(n):
        cluster_k_rows = chosen_trees[chosen_trees.cluster==k]
        cluster_k_trees = cluster_k_rows.apply(lambda x: trees[x.patient, x.good_tree], axis=1)
        center = centers[k]
        # cluster_k_distances[i] = distance between tree i and center
        cluster_k_distances = cluster_k_trees.apply(lambda x: DistanceLib.distance_profile(x, center), axis=1)
        wss += sum(cluster_k_distances)
    return wss

In [6]:
trees = ClusterLib.trees

n_range = range(1, 10)
wss = []
for i in n_range:
    chosen_trees, centers, converged = ClusterLib.tree_kmeans(i)
    wss.append(calculate_wss(i, chosen_trees, centers))

# plot wss vs. # of clusters
# choose the elbow as the "optimal" # of trees
plt.plot(n_range, wss)

In [ ]:
n = 2
chosen_trees, centers, converged = tree_kmeans(n)